# Lab-06 Softmax Classification
- Softmax
- Cross Entropy
- Low-level Implementation
- High-level Implementation
- Training Example

Logistic Regression의 연장선상이라고 이해해도 됨.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [38]:
# For repreducibility
torch.manual_seed(1)

## Discrete Probability Distribution

이산 확률 분포


$P(X=가위)=0.2$, $P(X=6)=\frac{1}{6}$


## Softmax

Convert numbers to probabilities with softmax.

max 값을 뽑아 주는데, soft하게 뽑아준다는 의미

$P(class=i)=\frac{e^i}{\sum{e^i}}$

In [3]:
z = torch.FloatTensor([1, 2, 3])

PyTorch has a `softmax` function

In [10]:
hypothesis = F.softmax(z, dim=0)
print(hypothesis)

tensor([0.0900, 0.2447, 0.6652])


원래는 $max=(0, 0, 1)$이어야 하겠지만 soft하게 뽑아주므로 $softmax=(0.0900, 0.2447, 0.6652)$

$(0.09 = \frac{e^1}{e^1 + e^2 + e^3})$


ex) 예전에 가위를 냈을 때 주먹을 낼 확률 -> $P(주먹 | 가위)$, $P(가위 | 가위)$, $P(보 | 가위)$

Since they are probabilities, they should add up to 1. Let's do a sanity check.

In [11]:
hypothesis.sum()   # 합은 1

tensor(1.)

For multi-class classification, we use the cross entropy loss.



## Cross Entropy

$H(P, Q) = -\mathbb{E}_{x \sim P(x)}[\log Q(x)] = - \sum_{x \in X}P(x) \log Q(x)$

Cross Entropy를 최소화하는 것(-> 0)이 중요하다.

## Cross Entropy Loss (Low-level)

For multi-class classification, we use the cross entropy loss.

$L = \frac{1}{N} \sum -y \log (\hat{y})$

y -> P(x) : 실제 y, y hat -> Q(x) : P_theta(x)

where $\hat{y}$ is the predicted probability and $y$ is the correct probability (0 or 1).

In [19]:
# #classes = 5, #samples = 3
z = torch.rand(3, 5, requires_grad=True)    # |z| = (3, 5), 그래디언트 학습하기
hypothesis = F.softmax(z, dim=1)            # prediction = y hat
print(hypothesis)

tensor([[0.1346, 0.2520, 0.1707, 0.2451, 0.1975],
        [0.2180, 0.2065, 0.2219, 0.2371, 0.1164],
        [0.2781, 0.1521, 0.1483, 0.2229, 0.1986]], grad_fn=<SoftmaxBackward0>)


In [18]:
y = torch.randint(5, (3,)).long()  # 따로 주어진 정답이 없으니, 정답을 랜덤하게 생성해 보자.
print(y)

tensor([1, 2, 2])


In [25]:
y_one_hot = torch.zeros_like(hypothesis)   # |y_one_hot| = (3, 5)
y_one_hot.scatter_(1, y.unsqueeze(1), 1)   # y.unsqueeze(1)을 dim=1로 1을 뿌려라! _는 in-place
# |y| = (3,) => |y.unsqueeze(1)| = (3, 1)
# y = [0, 2, 1] => y.unsqueeze(1) = [0, 2, 1]^T

tensor([[0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0.]])

In [27]:
cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()
# y_one_hot * -torch.log(hypothesis) => (3, 5)
# .sum(dim=1) => (3,)
# .mean() => scalar
print(cost)

tensor(1.5975, grad_fn=<MeanBackward0>)


## Cross-entropy Loss with torch.nn.functional

위의 것은 low-level이니 좀 더 편리한 모듈 제공

In [39]:
# High level
F.log_softmax(z, dim=1)    # Low level : torch.log(F.softmax(z, dim=1))

tensor([[-2.0052, -1.3784, -1.7676, -1.4061, -1.6218],
        [-1.5231, -1.5773, -1.5055, -1.4394, -2.1503],
        [-1.2797, -1.8830, -1.9087, -1.5010, -1.6166]],
       grad_fn=<LogSoftmaxBackward0>)

In [41]:
# Low level
#(y_one_hot * -torch.log(F.softmax(z, dim=1))).sum(dim=1).mean()

# High level
F.nll_loss(F.log_softmax(z, dim=1), y)   # NLL = Negative Log Likelihood

tensor(1.5975, grad_fn=<NllLossBackward0>)

**PyTorch also has `F.cross_entropy` that combines `F.log_softmax()` and `F.nll_loss()`!**

In [37]:
F.cross_entropy(z, y)

tensor(1.5975, grad_fn=<NllLossBackward0>)

## Training with Low-level Cross Entropy Loss

#samples = m, #classes = 3, dim = 4

In [43]:
x_train = [[1, 2, 1, 1],
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 5],
           [1, 6, 6, 6],
           [1, 7, 7, 7]]
y_train = [2, 2, 2, 1, 1, 1, 0, 0]

# |x_train| = (m, 4)
# |y_train| = (m,)

x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)

In [51]:
# 모델 초기화
W = torch.zeros((4, 3), requires_grad=True)   # (4, 3)이므로 4 samples -> 3 classes
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([W, b], lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    
    # Cost 계산
    hypothesis = F.softmax(x_train.matmul(W) + b, dim=1)   # or .mm or @
    y_one_hot = torch.zeros_like(hypothesis)
    y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)
    cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()
    
    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print("Epoch {:4d}/{} Cost: {:6f}".format(epoch, nb_epochs, cost.item()))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.775065
Epoch  200/1000 Cost: 0.712725
Epoch  300/1000 Cost: 0.677388
Epoch  400/1000 Cost: 0.650680
Epoch  500/1000 Cost: 0.627860
Epoch  600/1000 Cost: 0.607082
Epoch  700/1000 Cost: 0.587473
Epoch  800/1000 Cost: 0.568567
Epoch  900/1000 Cost: 0.550100
Epoch 1000/1000 Cost: 0.531914


## Training with F.cross_entropy

cross_entropy()를 쓰면 좀 더 간단해짐.

In [46]:
# 모델 초기화
W = torch.zeros((4, 3), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([W, b], lr=0.1)

nb_epochs = 1000                               # 1000번 epoch 돌리고
for epoch in range(nb_epochs + 1):
    
    # Cost 계산 (2)
    z = x_train.matmul(W) + b  # or .mm or @   # 여기까진 똑같다.
    cost = F.cross_entropy(z, y_train)         # 다만 cost 구할 때, z, y 통해 실제 정답 바로 비교. (scatter 필요 없음.)
                                               # 즉, one hot vector 필요 없다.
    
    # cost로 H(x) 계산
    optimizer.zero_grad()                      # optimizer에 zero grad를 하고,
    cost.backward()                            # back propagation하고,
    optimizer.step()                           # back prop.해서 얻은 gradient대로 다시 한번 grad. descent 통해서
                                               # cross entropy 함수를 minimize하돌고 한다.
                                               # minimize해서 어떤 grad truth 확률 분포에 좀 더 근사!
    
    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print("Epoch {:4d}/{} Cost: {:.6f}".format(epoch, nb_epochs, cost.item()))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.775066
Epoch  200/1000 Cost: 0.712725
Epoch  300/1000 Cost: 0.677388
Epoch  400/1000 Cost: 0.650680
Epoch  500/1000 Cost: 0.627860
Epoch  600/1000 Cost: 0.607082
Epoch  700/1000 Cost: 0.587473
Epoch  800/1000 Cost: 0.568567
Epoch  900/1000 Cost: 0.550100
Epoch 1000/1000 Cost: 0.531914


## High-level Implementation with `nn.Module`

In [49]:
class SoftmaxClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(4, 3)   # Output이 3! (4 => 3)
    
    def forward(self, x):
        return self.linear(x)           # |x| = (m, 4) => output (m, 3) 

In [48]:
model = SoftmaxClassifierModel()

In [52]:
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=0.1)  # 모델의 파라미터 즉 linear 레이어 하나 들어있는 건데 그걸 SGD

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    
    # H(x) 계산
    prediction = model(x_train)     # model에 통과. in: |x_train| = (m, 4) => out: |prediction| = (m, 3)
    
    # cost 계산
    cost = F.cross_entropy(prediction, y_train)  # |y_train| = (m,) (인덱스 값, 즉 아직 원핫벡터로 바꾸지 않은 상태)
                                                 # cost 계산하면 어떤 스칼라 값 나올 것.
    
    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()         # back propagation 수행하면 gradient descent가 model.parameters()에 채워졌겠지
    optimizer.step()        # 그러니 우리는 거기에 대해 gradient descent 1 step을 수행. 이걸 1000번 수행
    
    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print("Epoch {:4d}/{} Cost: {:.6f}".format(epoch, nb_epochs, cost.item()))

Epoch    0/1000 Cost: 1.626739
Epoch  100/1000 Cost: 0.664001
Epoch  200/1000 Cost: 0.582189
Epoch  300/1000 Cost: 0.533072
Epoch  400/1000 Cost: 0.494812
Epoch  500/1000 Cost: 0.461731
Epoch  600/1000 Cost: 0.431563
Epoch  700/1000 Cost: 0.403123
Epoch  800/1000 Cost: 0.375636
Epoch  900/1000 Cost: 0.348516
Epoch 1000/1000 Cost: 0.321313


**보면 알겠지만, softmax classification은 logistic regression과 굉장히 유사.**

logi. regr.는 0과 1의 두 가지 class만 있는 discrete한 class가 있는 classification (binary classification),
softmax classification의 경우는, multinary, multinomial한 확률 분포가 되겠다.

그래서 추후 딥러닝할 때 보통 습관적으로 어떤 classification을 하기 위해서는, cross entropy를 써야 하고,
softmax layer를 쓰게 되기 마련인데, 사실, binary classification인 경우의 문제도 많다.

그럴 때는 logistic regression에서 사용한 loss 함수를 사용하는 것이 맞다.
즉, binary cross entropy (BCE)를 사용하는 것이 맞다. 그리고 sigmoid를 사용해야겠지.

즉,

- binary classification
    - BCE
    - sigmoid
    
- multinary classification (지금처럼 여러 개의 클래스가 있을 때)
    - CE
    - softmax layer
    
이러한 부분에 있어 추후 딥러닝을 직접 구현할 때 있어 주의하기 바람.